In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [3]:

mnist = input_data.read_data_sets("MINIST_data",one_hot=True)

batch_size = 50

n_batch = mnist.train.num_examples // batch_size

#参赛概要

def variable_summary(var):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean',mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev',stddev)
    tf.summary.scalar('max',tf.reduce_max(var))
    tf.summary.scalar('min',tf.reduce_min(var))
    tf.summary.histogram('histogram',var)

with tf.device('/device:GPU:0'):
    #命名空间
    #placeholder
    with tf.name_scope("input"):
        x = tf.placeholder(tf.float32,[None,784],name = "x-input")
        y = tf.placeholder(tf.float32,[None,10],name = "y-input")

        #创建简单神经网络
    with tf.name_scope("layer"):
        with tf.name_scope("weight"):  
            w = tf.Variable(tf.zeros([784,10]),name='w')
            variable_summary(w)
        with tf.name_scope("biases"):
            b = tf.Variable(tf.zeros([10])+0.01,name='b')
            variable_summary(b)
        with tf.name_scope("softmax"):

            prediction = tf.nn.softmax(tf.matmul(x,w)+b)

    #二次代价函数
    #loss = tf.reduce_mean(tf.square(y - prediction))
    #交叉熵
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,logits = prediction))
        tf.summary.scalar('loss',loss)

    #优化器
    with tf.name_scope("train"):

        #train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
        #train_step = tf.train.AdadeltaOptimizer(0.02).minimize(loss)

    init = tf.global_variables_initializer()

    with tf.name_scope("accuracy"):
        with tf.name_scope("correct_prediction"):
            correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
        with tf.name_scope("correct_prediction"):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
            tf.summary.scalar('accuracy',accuracy)

    #合并所有的summary
    merged = tf.summary.merge_all()

        
s = time.time()
with tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True)) as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("logs",sess.graph)
    for epoch in range(20):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict = {x:batch_xs,y:batch_ys})
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels})
        
        print("Iter " + str(epoch) + ",Testing accuracy " + str(acc))
print(time.time() - s)

Extracting MINIST_data\train-images-idx3-ubyte.gz
Extracting MINIST_data\train-labels-idx1-ubyte.gz
Extracting MINIST_data\t10k-images-idx3-ubyte.gz
Extracting MINIST_data\t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'input/x-input' with dtype float and shape [?,784]
	 [[node input/x-input (defined at <ipython-input-2-f85400cee1b0>:23) ]]
	 [[{{node _arg_input_1/x-input_0_0}}]]

Caused by op 'input/x-input', defined at:
  File "C:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 148, in sta